In [1]:
#Uploading relevant libraries 
import pandas as pd
import seaborn as sns
sns.set()
!pip install plotly
from IPython.display import IFrame
import scipy as sp
import plotly.express as px
!pip install jupyterlab "ipywidgets>=7.5"
import matplotlib.pyplot as plt
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import word_tokenize

import nltk
from nltk.stem import WordNetLemmatizer
from nltk import stem
stemmer = stem.PorterStemmer()
from nltk.corpus import stopwords
nltk.download('stopwords')
stops = set(stopwords.words('english'))
import string
punct = list(string.punctuation)
from collections import Counter
import requests
import numpy as np
import praw 
import datetime
import random
from random import sample

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nicolerodriguezq/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# About This Notebook

## Corpus of Data: Reddit's Top Upvoted Stories
This notebook shows the collection, cleaning and preprocessing of corpus of data from the Subreddit **‘Stories’**, a top 1% community in Reddit. I gathered the top 10 stories of all time, alongside the comments made to each post, aiming to have a ready-to-use linguistic dataset for future NLP (Natural Language Processing) analysis of narratives, storytelling, and public engagement.

#### NLP

It’s the use of linguistics, statistics and machine learning through computational modelling, that allows us to study insights on various aspects of human languages, such as (but not limited to) interactions, emotions, and machine translation (Joshi, 1991).

#### What Makes a Good Story?

Specifically, I aim to explore the the linguistic characteristics of highly upvoted and commented stories, along with the emotional dimensions of both the narratives and the people who engage with them. 

--------------

In [2]:
#uploading datasets to index later.
vad = pd.read_excel('vad.xlsx', index_col =0)
sm = pd.read_excel('sensorimotor.xlsx', index_col =0)

### API (Application Programming Interface)

To gather all the reddit posts, I used APIs - specific functions that allows a one way ‘transaction’ within two applications (Frye, 2018), in this case me, in this notebook, and Reddit. The APIs allowed me to extract the stories and their metadata directly to this notebook.

In [4]:
#reddit set-up
reddit = praw.Reddit(user_agent='VAD',
                    client_id='6K4b0HkPXk1fmP7kDc1-gQ', client_secret= "Jux1YRp1VJwHzb8yZC-X0lr9QwOBFg",
                    username= 'NicolleVR', password= '/dk@4^u#Tw%J65:')

In [5]:
#function to extract specific data needed
def submission(submission_id):
    try:
        submission = reddit.submission(url=submission_id)
    except:
        submission = reddit.submission(submission_id)

    # Extracting main post details
    main_post = {
        'text': submission.selftext,
        'datetime': datetime.datetime.fromtimestamp(submission.created_utc),
        'score': submission.score,
        'subreddit': submission.subreddit,
        'redditor': submission.author,
        'type': 'submission',
        'title': submission.title
    }

    # Extracting comments details
    submission.comments.replace_more()
    comments = []
    for comment in submission.comments.list():
        comment_data = {
            'text': comment.body,
            'datetime': datetime.datetime.fromtimestamp(comment.created_utc),
            'score': comment.score,
            'subreddit': submission.subreddit,
            'redditor': comment.author,
            'type': 'comment',
            'title': submission.title
        }
        comments.append(comment_data)

    # Concatenate main post and comments into a DataFrame
    df = pd.DataFrame([main_post] + comments)

    # Sorting DataFrame by score
    df = df.sort_values('score', ascending=False).reset_index(drop=True)

    return df

In [6]:
#checking the code works with the first top story.
story_1 = submission('https://www.reddit.com/r/stories/comments/16vqxs6/i_thought_she_took_me_to_the_restroom_to_have_sex/')

In [7]:
story_1.head()

,text,datetime,score,subreddit,redditor,type,title
0,In college I met a girl at a house party. We d...,2023-09-30 00:25:52,37468,stories,CardiffGiant1212,submission,I thought she took me to the restroom to have ...
1,"Dude, I hate to break it to you but as soon as...",2023-09-30 01:34:12,1779,stories,stpg1222,comment,I thought she took me to the restroom to have ...
2,Coward,2023-09-30 00:38:37,1226,stories,Tall-Remote3112,comment,I thought she took me to the restroom to have ...
3,Lmao I asked my husband what he would do if th...,2023-09-30 02:13:34,1055,stories,Jilltro,comment,I thought she took me to the restroom to have ...
4,"Everyone knows the order is drinking, talking,...",2023-09-30 01:24:44,715,stories,oldmanartie,comment,I thought she took me to the restroom to have ...


In [8]:
#running the other 9 top stories (most upvoted stories)
story_2 = submission('https://www.reddit.com/r/stories/comments/15ysuej/i32m_am_divorcing_my_wife_33f_after_finding_out/')
story_3 = submission('https://www.reddit.com/r/stories/comments/1612ts0/my_sister_died_during_postpartum_and_im_a/')
story_4 = submission('https://www.reddit.com/r/stories/comments/15skbuq/i_surprised_my_girlfriend_with_taylor_swift/')
story_5 = submission('https://www.reddit.com/r/stories/comments/16ezv93/i_accidentally_shit_the_bed_in_front_of_my/')
story_6 = submission('https://www.reddit.com/r/stories/comments/18d8tvj/my_girlfriend_confessed_to_cheating_on_me_so_i/')
story_7 = submission('https://www.reddit.com/r/stories/comments/17gkyty/a_quick_story_about_my_dating_policy_for_my/')
story_8 = submission('https://www.reddit.com/r/stories/comments/16piw5d/four_days_ago_my_coworker_whos_also_my_friend/')
story_9 = submission('https://www.reddit.com/r/stories/comments/1837bm4/my_boyfriend_turned_me_bi/')
story_10 = submission('https://www.reddit.com/r/stories/comments/15gzmzb/husband_wants_to_reset_his_whole_life/')

In [9]:
#making a single dataset out of all stories. 
all_stories = pd.concat([story_1, story_2, story_3, story_4, story_5, story_6, story_7,\
                  story_8, story_9, story_10], ignore_index=True)

## Saving the Data

In [10]:
#creating a csv file with the dataset to not run the APIs again. 
all_stories.to_csv('top_10_stories.csv')
all_stories = pd.read_csv('top_10_stories.csv', index_col = 0)

In [11]:
all_stories.head()

,text,datetime,score,subreddit,redditor,type,title
0,In college I met a girl at a house party. We d...,2023-09-30 00:25:52,37468,stories,CardiffGiant1212,submission,I thought she took me to the restroom to have ...
1,"Dude, I hate to break it to you but as soon as...",2023-09-30 01:34:12,1779,stories,stpg1222,comment,I thought she took me to the restroom to have ...
2,Coward,2023-09-30 00:38:37,1226,stories,Tall-Remote3112,comment,I thought she took me to the restroom to have ...
3,Lmao I asked my husband what he would do if th...,2023-09-30 02:13:34,1055,stories,Jilltro,comment,I thought she took me to the restroom to have ...
4,"Everyone knows the order is drinking, talking,...",2023-09-30 01:24:44,715,stories,oldmanartie,comment,I thought she took me to the restroom to have ...


## Datasets 
I decided to create two datasets, one only with the stories posted, one with the comments. This is to be able to analyse them separately. For instance, I could explore the emotional dimensions the stories have, and whether they provoke similar, or different emotions to the readers. 

In [12]:
#DataFrame only with the comments 
df_comments = all_stories[all_stories['type'] == 'comment']
df_comments.reset_index(drop=True, inplace=True)
df_comments.head()

,text,datetime,score,subreddit,redditor,type,title
0,"Dude, I hate to break it to you but as soon as...",2023-09-30 01:34:12,1779,stories,stpg1222,comment,I thought she took me to the restroom to have ...
1,Coward,2023-09-30 00:38:37,1226,stories,Tall-Remote3112,comment,I thought she took me to the restroom to have ...
2,Lmao I asked my husband what he would do if th...,2023-09-30 02:13:34,1055,stories,Jilltro,comment,I thought she took me to the restroom to have ...
3,"Everyone knows the order is drinking, talking,...",2023-09-30 01:24:44,715,stories,oldmanartie,comment,I thought she took me to the restroom to have ...
4,This reminds me of college when my roommates g...,2023-09-30 02:19:20,541,stories,talon2525,comment,I thought she took me to the restroom to have ...


In [13]:
#separating the posts from the comments 
df_posts = all_stories[all_stories['type'] == 'submission'].reset_index(drop=True)
df_posts

,text,datetime,score,subreddit,redditor,type,title
0,In college I met a girl at a house party. We d...,2023-09-30 00:25:52,37468,stories,CardiffGiant1212,submission,I thought she took me to the restroom to have ...
1,"Well, a couple of weeks ago I found out that m...",2023-08-23 05:12:17,15814,stories,OkDot3924,submission,I(32M) am divorcing my wife (33F) after findin...
2,My 28 year old sister gave birth to her first ...,2023-08-25 16:44:06,14396,stories,dearSisterLove,submission,My sister died during post-partum and I'm a co...
3,"me and my girlfriend,(both 26) have been datin...",2023-08-16 09:54:26,12598,stories,Then-Tale3612,submission,I surprised my girlfriend with Taylor swift ti...
4,WARNING! don't read this if you have any probl...,2023-09-10 14:07:01,11581,stories,missemusepigen,submission,"I ""accidentally"" shit the bed in front of my b..."
5,My(25m) girlfriend(26) confessed to cheating o...,2023-12-07 23:29:30,10885,stories,disheveledhalfwit,submission,My girlfriend confessed to cheating on me so I...
6,"When my daughter was 5 years old, and just sta...",2023-10-26 02:29:16,10313,stories,itsathrowaway6877,submission,A quick story about my dating policy for my da...
7,Four days ago I was walking with a coworker on...,2023-09-22 20:21:09,10136,stories,Sheeitsheeit,submission,Four days ago my coworker who's also my friend...
8,Edit: Reading through the comments made me rea...,2023-11-25 00:56:13,9477,stories,Suspicious-Group-418,submission,My boyfriend turned me bi
9,"Hi, I'm a 35 year old woman married to a 45 ye...",2023-08-03 10:50:11,9377,stories,Witty-Firefighter-28,submission,Husband wants to reset his whole life.


## Tokens and Lemmas
Tokenisation converts texts into individual words, and lemmatising converts them into their base  (Educative, 2024) E.g. Mice to mouse. These are functions within an NLP library - NLKT (Natural Language Toolkit). 

In [14]:
#lemmatise and tokenise function
def lemma(text):
    tokens = word_tokenize(text)
    lemmas = [lemmatizer.lemmatize(i.lower()) for i in tokens if i.lower() not in stops and i.lower() not in punct]
    return lemmas

In [15]:
#Checking it works 
lemma('The Spiders, dog and mice.')

['spider', 'dog', 'mouse']

In [17]:
#Applying lemma function to both dataframes 
df_posts.loc[:, 'text_lemmatised'] = df_posts['text'].apply(lemma)
df_comments.loc[:, 'text_lemmatised'] = df_comments['text'].apply(lemma)

I applied the 'lemma' function as a separate column in both dataframes as I want to have access to the original text, in case this is of any use. Though this is still unclear. 

## VAD Dataset 
VAD is a model of emotions. It quantifies individual words with a specific value on three dimensions: valence, arousal and dominance. These define positivity and negativity, excitement and relaxation, and feeling in control or without it, respectively (Jones, Libert and Tynski, 2016)

In [18]:
#adding the VAD scores to the datasets 
vad_scores_posts = []

for lemmas in df_posts['text_lemmatised']:
    vad_words = []
    try:
        for word in lemmas:
            if word.lower() in vad.index:
                vad_words.append(word.lower())
        vad_ = vad.loc[vad_words]
        vad_scores_posts.append(vad_.mean())
    except:
        vad_scores_posts.append(pd.Series([np.nan for _ in range(3)]))

#DataFrame from vad_scores_posts
vad_scores_posts_df = pd.DataFrame(vad_scores_posts, columns=['valence', 'arousal', 'dominance'])

#Concatenate vad_scores_posts_df with the original 'df_posts' dataframe
df_posts = pd.concat([df_posts, vad_scores_posts_df], axis=1)


In [19]:
vad_scores_comments = []

for lemmas in df_comments['text_lemmatised']:
    vad_words = []
    try:
        for word in lemmas:
            if word.lower() in vad.index:
                vad_words.append(word.lower())
        vad_ = vad.loc[vad_words]
        vad_scores_comments.append(vad_.mean())
    except:
        vad_scores_comments.append(pd.Series([np.nan for _ in range(3)]))

#DataFrame from vad_scores_comments
vad_scores_comments_df = pd.DataFrame(vad_scores_comments, columns=['valence', 'arousal', 'dominance'])

#Concatenate vad_scores_comments_df with the original 'df_comments' dataframe
df_comments = pd.concat([df_comments, vad_scores_comments_df], axis=1)


The datasets are ready. However, I will save them in the folder to access them easier. 

In [20]:
df_comments.to_csv('comments.csv')
df_comments = pd.read_csv('comments.csv', index_col = 0)
df_posts.to_csv('posts.csv')
df_posts = pd.read_csv('posts.csv', index_col = 0)

In [24]:
df_posts.head()

,text,datetime,score,subreddit,redditor,type,title,text_lemmatised,valence,arousal,dominance
0,In college I met a girl at a house party. We d...,2023-09-30 00:25:52,37468,stories,CardiffGiant1212,submission,I thought she took me to the restroom to have ...,"['college', 'met', 'girl', 'house', 'party', '...",0.619743,0.496730,0.611576
1,"Well, a couple of weeks ago I found out that m...",2023-08-23 05:12:17,15814,stories,OkDot3924,submission,I(32M) am divorcing my wife (33F) after findin...,"['well', 'couple', 'week', 'ago', 'found', 'wi...",0.593527,0.510437,0.558512
2,My 28 year old sister gave birth to her first ...,2023-08-25 16:44:06,14396,stories,dearSisterLove,submission,My sister died during post-partum and I'm a co...,"['28', 'year', 'old', 'sister', 'gave', 'birth...",0.579823,0.509153,0.529868
3,"me and my girlfriend,(both 26) have been datin...",2023-08-16 09:54:26,12598,stories,Then-Tale3612,submission,I surprised my girlfriend with Taylor swift ti...,"['girlfriend', '26', 'dating', 'three', 'year'...",0.676503,0.532078,0.610264
4,WARNING! don't read this if you have any probl...,2023-09-10 14:07:01,11581,stories,missemusepigen,submission,"I ""accidentally"" shit the bed in front of my b...","['warning', ""n't"", 'read', 'problem', 'feces',...",0.611488,0.518045,0.593588


In [25]:
df_comments.head()

,text,datetime,score,subreddit,redditor,type,title,text_lemmatised,valence,arousal,dominance
0,"Dude, I hate to break it to you but as soon as...",2023-09-30 01:34:12,1779,stories,stpg1222,comment,I thought she took me to the restroom to have ...,"['dude', 'hate', 'break', 'soon', 'farted', 'f...",0.543494,0.640114,0.529571
1,Coward,2023-09-30 00:38:37,1226,stories,Tall-Remote3112,comment,I thought she took me to the restroom to have ...,['coward'],0.289720,0.463122,0.387136
2,Lmao I asked my husband what he would do if th...,2023-09-30 02:13:34,1055,stories,Jilltro,comment,I thought she took me to the restroom to have ...,"['lmao', 'asked', 'husband', 'would', 'happene...",0.548481,0.481526,0.575000
3,"Everyone knows the order is drinking, talking,...",2023-09-30 01:24:44,715,stories,oldmanartie,comment,I thought she took me to the restroom to have ...,"['everyone', 'know', 'order', 'drinking', 'tal...",0.671437,0.488964,0.617617
4,This reminds me of college when my roommates g...,2023-09-30 02:19:20,541,stories,talon2525,comment,I thought she took me to the restroom to have ...,"['reminds', 'college', 'roommate', 'gf', 'came...",0.613123,0.491981,0.567193


In [26]:
len(df_comments)

22263

In [27]:
len(df_posts)

10

## Limitations 
Whilst I hope that this corpus could bring some insights in storytelling, I am aware that only sourcing data from Reddit might not be representative of all ‘successful’ storytelling. More studies across different platforms would be needed to clarify that. Also, I must admit that the ‘df_stories; data frame might be too small to provide a good view on sentiment analysis trends, or narrative structures. 

Furthermore, to have a more holistic view on narratives and storytelling, less popular stories should be included in the corpus. However, Reddit imposes a big limitation, at least through APIs, as posts with downvotes or less comments are not easily accessible, meaning there isn’t an option to filter them. If I want to include them, I would have to source the data manually, which is time consuming and the collection would be subject to my own biases. 

### NLP Limitations
Even if the limitations above were not present, it’s also worth mentioning that NLP in itself presents limitations, such as difficulties accessing APIs from more renowned social media platforms that could benefit the analysis, and loss in semantics through the applications of lemmas, tokens and VAD values.

------------
#### This work contains 559 words. 
-------------

#### Declarations
I have used chat GPT to work on this assignment mainly to check the code was written in the right format, for instance, using using the correct brackets for indexing or applying functions. I did not copy a paste any code which was solely written by AI. I also checked my overall idea in storytelling, but I did not find this of much help. 


# Bibliography 
    Educative (2024). Difference between tokenization and lemmatization in NLP. [online] Educative. Available at: https://www.educative.io/answers/difference-between-tokenization-and-lemmatization-in-nlp.Frye

    M.-K. (2018). What is an API? (Application Programming Interface). [online] MuleSoft. Available at: https://www.mulesoft.com/resources/api/what-is-an-api.IBM 
    
    (n.d.). What is Natural Language Processing? | IBM. [online] www.ibm.com. Available at: https://www.ibm.com/topics/natural-language-processing#:~:text=the%20next%20step-.Jones
    
    K., Libert, K. and Tynski, K. (2016). The Emotional Combinations That Make Stories Go Viral. [online] Harvard Business Review. Available at: https://hbr.org/2016/05/research-the-link-between-feeling-in-control-and-viral-content#:~:text=The%20findings%20indicate%20that%20individual.Joshi
    
    A.K. (1991). Natural Language Processing. Science, 253(5025), pp.1242–1249. doi:https://doi.org/10.1126/science.253.5025.1242.
